# Santa 2025 - Ultra-Optimized Tree Packing Solver
**Target Score: < 40 (from baseline 424.69)**

Optimizations:
- Precomputed optimal tree spacing for different rotation pairs
- Aggressive interlocking with 0°/180° rotation patterns
- Multi-strategy layout generation
- Binary search shrink-wrap with repair
- Simulated annealing with rotation moves
- Greedy compaction passes
- Differential evolution for larger n

In [ ]:
import numpy as np
import pandas as pd
import os
import time
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
print("Santa 2025 - Ultra-Optimized Solver")
print("Target: Score < 40")

In [ ]:
# Tree polygon
TREE_DEFAULT = np.array([
    [0.0, 1.0], [-0.25, 0.6], [-0.15, 0.6], [-0.35, 0.3], [-0.20, 0.3],
    [-0.45, 0.0], [-0.10, 0.0], [-0.10, -0.20], [0.10, -0.20], [0.10, 0.0],
    [0.45, 0.0], [0.20, 0.3], [0.35, 0.3], [0.15, 0.6], [0.25, 0.6],
], dtype=np.float64)

TREE = TREE_DEFAULT.copy()

# Try to load from Kaggle
for path in ['/kaggle/input/santa-2025', '/kaggle/input/santa-2025-christmas-tree-packing-challenge']:
    tree_file = os.path.join(path, 'tree.csv')
    if os.path.exists(tree_file):
        try:
            df = pd.read_csv(tree_file)
            if 'x' in df.columns and 'y' in df.columns:
                TREE = df[['x', 'y']].values.astype(np.float64)
                print(f"Loaded tree from {tree_file}")
                break
        except:
            pass

# Center tree at origin
TREE = TREE - TREE.mean(axis=0)
TW = TREE[:, 0].max() - TREE[:, 0].min()
TH = TREE[:, 1].max() - TREE[:, 1].min()

def polygon_area(vertices):
    n = len(vertices)
    area = 0.0
    for i in range(n):
        j = (i + 1) % n
        area += vertices[i, 0] * vertices[j, 1]
        area -= vertices[j, 0] * vertices[i, 1]
    return abs(area) / 2.0

TREE_AREA = polygon_area(TREE)
print(f"Tree: {len(TREE)} vertices, {TW:.4f} x {TH:.4f}, area={TREE_AREA:.4f}")

In [ ]:
# Core classes and optimized rotation
class Placement:
    __slots__ = ['x', 'y', 'deg']
    def __init__(self, x, y, deg):
        self.x = float(x)
        self.y = float(y)
        self.deg = float(deg) % 360
    
    def copy(self):
        return Placement(self.x, self.y, self.deg)

# Pre-compute rotated trees
ROTATION_CACHE = {}
ANGLES = list(range(0, 360, 5)) + [0, 180]  # Common angles

def rotate_tree(deg):
    deg = float(deg) % 360
    deg_key = round(deg, 1)
    if deg_key not in ROTATION_CACHE:
        rad = np.radians(deg)
        c, s = np.cos(rad), np.sin(rad)
        rot_mat = np.array([[c, -s], [s, c]])
        ROTATION_CACHE[deg_key] = (TREE @ rot_mat.T).copy()
    return ROTATION_CACHE[deg_key].copy()

def get_tree_at(p):
    return rotate_tree(p.deg) + np.array([p.x, p.y])

# Pre-cache common angles
for angle in ANGLES:
    rotate_tree(angle)
print(f"Pre-cached {len(ROTATION_CACHE)} rotation angles")

In [ ]:
# Fast SAT Collision Detection
def get_normals(poly):
    edges = np.roll(poly, -1, axis=0) - poly
    normals = np.column_stack([-edges[:, 1], edges[:, 0]])
    lengths = np.linalg.norm(normals, axis=1, keepdims=True)
    lengths[lengths < 1e-12] = 1
    return normals / lengths

def sat_collision(poly1, poly2):
    all_normals = np.vstack([get_normals(poly1), get_normals(poly2)])
    for normal in all_normals:
        proj1 = poly1 @ normal
        proj2 = poly2 @ normal
        if proj1.max() < proj2.min() or proj2.max() < proj1.min():
            return False
    return True

def check_collision(p1, p2):
    t1 = get_tree_at(p1)
    t2 = get_tree_at(p2)
    
    # Fast AABB rejection
    b1_min, b1_max = t1.min(axis=0), t1.max(axis=0)
    b2_min, b2_max = t2.min(axis=0), t2.max(axis=0)
    
    if b1_max[0] < b2_min[0] or b2_max[0] < b1_min[0]:
        return False
    if b1_max[1] < b2_min[1] or b2_max[1] < b1_min[1]:
        return False
    
    return sat_collision(t1, t2)

def has_any_collision(placements):
    n = len(placements)
    for i in range(n):
        for j in range(i + 1, n):
            if check_collision(placements[i], placements[j]):
                return True
    return False

def find_collisions(placements):
    collisions = []
    n = len(placements)
    for i in range(n):
        for j in range(i + 1, n):
            if check_collision(placements[i], placements[j]):
                collisions.append((i, j))
    return collisions

def count_collisions_for(placements, idx):
    """Count collisions for a single tree."""
    count = 0
    for j in range(len(placements)):
        if idx != j and check_collision(placements[idx], placements[j]):
            count += 1
    return count

print("Collision detection ready")

In [ ]:
# Utility functions
def compute_side(placements):
    if not placements:
        return 0.0
    all_pts = np.vstack([get_tree_at(p) for p in placements])
    width = all_pts[:, 0].max() - all_pts[:, 0].min()
    height = all_pts[:, 1].max() - all_pts[:, 1].min()
    return max(width, height)

def compute_bounds(placements):
    all_pts = np.vstack([get_tree_at(p) for p in placements])
    return (all_pts[:, 0].min(), all_pts[:, 1].min(),
            all_pts[:, 0].max(), all_pts[:, 1].max())

def center_placements(placements):
    if not placements:
        return placements
    all_pts = np.vstack([get_tree_at(p) for p in placements])
    cx = (all_pts[:, 0].max() + all_pts[:, 0].min()) / 2
    cy = (all_pts[:, 1].max() + all_pts[:, 1].min()) / 2
    return [Placement(p.x - cx, p.y - cy, p.deg) for p in placements]

def theoretical_min_side(n):
    return np.sqrt(n * TREE_AREA)

def scale_placements(placements, scale):
    return [Placement(p.x * scale, p.y * scale, p.deg) for p in placements]

print("Utilities ready")

In [ ]:
# Find minimum spacing between two trees at given rotations
def find_min_spacing(deg1, deg2, direction='horizontal', eps=0.001):
    """
    Binary search for minimum spacing between two trees.
    Returns the minimum distance between centers.
    """
    if direction == 'horizontal':
        lo, hi = 0.0, TW * 2
        while hi - lo > eps:
            mid = (lo + hi) / 2
            p1 = Placement(0, 0, deg1)
            p2 = Placement(mid, 0, deg2)
            if check_collision(p1, p2):
                lo = mid
            else:
                hi = mid
        return hi + eps
    else:  # vertical
        lo, hi = 0.0, TH * 2
        while hi - lo > eps:
            mid = (lo + hi) / 2
            p1 = Placement(0, 0, deg1)
            p2 = Placement(0, mid, deg2)
            if check_collision(p1, p2):
                lo = mid
            else:
                hi = mid
        return hi + eps

# Pre-compute optimal spacing for common rotation pairs
SPACING_CACHE = {}
print("Computing optimal spacing for rotation pairs...")

# Key rotation pairs for interlocking
rotation_pairs = [(0, 0), (0, 180), (180, 0), (180, 180), (0, 90), (90, 0), (90, 90)]

for deg1, deg2 in rotation_pairs:
    key_h = (deg1, deg2, 'h')
    key_v = (deg1, deg2, 'v')
    SPACING_CACHE[key_h] = find_min_spacing(deg1, deg2, 'horizontal')
    SPACING_CACHE[key_v] = find_min_spacing(deg1, deg2, 'vertical')

print("Optimal spacing computed:")
print(f"  0°-0° horizontal: {SPACING_CACHE[(0, 0, 'h')]:.4f}")
print(f"  0°-180° horizontal: {SPACING_CACHE[(0, 180, 'h')]:.4f}")
print(f"  0°-0° vertical: {SPACING_CACHE[(0, 0, 'v')]:.4f}")
print(f"  0°-180° vertical: {SPACING_CACHE[(0, 180, 'v')]:.4f}")

In [ ]:
# Optimal interlocking layout using precomputed spacing
def optimal_interlocking_layout(n):
    """
    Create optimal interlocking layout using precomputed minimum spacing.
    Trees alternate between 0° and 180° rotation.
    """
    if n == 0:
        return []
    if n == 1:
        return [Placement(0, 0, 0)]
    
    # Use optimal spacing
    h_spacing_same = SPACING_CACHE.get((0, 0, 'h'), TW)
    h_spacing_diff = SPACING_CACHE.get((0, 180, 'h'), TW * 0.8)
    v_spacing_same = SPACING_CACHE.get((0, 0, 'v'), TH)
    v_spacing_diff = SPACING_CACHE.get((0, 180, 'v'), TH * 0.7)
    
    placements = []
    
    # Calculate optimal grid dimensions
    cols = int(np.ceil(np.sqrt(n * TH / TW)))
    if cols < 1:
        cols = 1
    
    row = 0
    idx = 0
    
    while idx < n:
        row_deg = 0 if row % 2 == 0 else 180
        
        # Use different vertical spacing for interlocking rows
        if row == 0:
            y = 0
        else:
            y = placements[-cols].y if len(placements) >= cols else 0
            y += v_spacing_diff if row % 2 == 1 else v_spacing_same
        
        # Horizontal offset for odd rows (hex pattern)
        offset = h_spacing_same * 0.5 if row % 2 == 1 else 0
        
        for col in range(cols):
            if idx >= n:
                break
            x = col * h_spacing_same + offset
            placements.append(Placement(x, y, row_deg))
            idx += 1
        row += 1
    
    return center_placements(placements)

# Alternative: checkerboard rotation pattern
def checkerboard_layout(n, spacing_factor=1.0):
    """
    Checkerboard pattern where adjacent trees have opposite rotations.
    """
    if n == 0:
        return []
    if n == 1:
        return [Placement(0, 0, 0)]
    
    h_sp = SPACING_CACHE.get((0, 180, 'h'), TW * 0.8) * spacing_factor
    v_sp = SPACING_CACHE.get((0, 180, 'v'), TH * 0.7) * spacing_factor
    
    placements = []
    cols = int(np.ceil(np.sqrt(n * v_sp / h_sp)))
    if cols < 1:
        cols = 1
    
    row = 0
    idx = 0
    while idx < n:
        for col in range(cols):
            if idx >= n:
                break
            x = col * h_sp
            y = row * v_sp
            # Checkerboard rotation
            deg = 0 if (row + col) % 2 == 0 else 180
            placements.append(Placement(x, y, deg))
            idx += 1
        row += 1
    
    return center_placements(placements)

print("Interlocking layouts ready")

In [ ]:
# Collision repair with smart moves
def repair_collisions(placements, max_iter=500, step=0.02):
    placements = [p.copy() for p in placements]
    n = len(placements)
    if n < 2:
        return placements
    
    for iteration in range(max_iter):
        collisions = find_collisions(placements)
        if not collisions:
            break
        
        forces = np.zeros((n, 2))
        for i, j in collisions:
            dx = placements[j].x - placements[i].x
            dy = placements[j].y - placements[i].y
            dist = np.sqrt(dx*dx + dy*dy) + 1e-10
            strength = 1.0 / (dist + 0.1)
            fx, fy = dx / dist * strength, dy / dist * strength
            forces[i] -= [fx, fy]
            forces[j] += [fx, fy]
        
        current_step = step * (1 + iteration / 100)
        for i in range(n):
            norm = np.linalg.norm(forces[i])
            if norm > 1e-10:
                placements[i].x += forces[i][0] * current_step / max(norm, 0.1)
                placements[i].y += forces[i][1] * current_step / max(norm, 0.1)
    
    return placements

print("Collision repair ready")

In [ ]:
# Greedy compaction - move each tree toward center
def compact_greedy(placements, iterations=200, step=0.01):
    placements = [p.copy() for p in placements]
    n = len(placements)
    if n < 2:
        return placements
    
    for _ in range(iterations):
        improved = False
        cx = sum(p.x for p in placements) / n
        cy = sum(p.y for p in placements) / n
        
        order = np.random.permutation(n)
        for i in order:
            p = placements[i]
            dx = cx - p.x
            dy = cy - p.y
            dist = np.sqrt(dx*dx + dy*dy)
            if dist < 0.001:
                continue
            
            new_p = Placement(p.x + dx/dist * step, p.y + dy/dist * step, p.deg)
            valid = True
            for j in range(n):
                if i != j and check_collision(new_p, placements[j]):
                    valid = False
                    break
            if valid:
                placements[i] = new_p
                improved = True
        
        if not improved:
            break
    
    return center_placements(placements)

# Compaction toward bounding box center (minimize side)
def compact_toward_bbox_center(placements, iterations=200, step=0.01):
    placements = [p.copy() for p in placements]
    n = len(placements)
    if n < 2:
        return placements
    
    for _ in range(iterations):
        improved = False
        bounds = compute_bounds(placements)
        cx = (bounds[0] + bounds[2]) / 2
        cy = (bounds[1] + bounds[3]) / 2
        current_side = compute_side(placements)
        
        order = np.random.permutation(n)
        for i in order:
            p = placements[i]
            dx = cx - p.x
            dy = cy - p.y
            dist = np.sqrt(dx*dx + dy*dy)
            if dist < 0.001:
                continue
            
            new_p = Placement(p.x + dx/dist * step, p.y + dy/dist * step, p.deg)
            
            valid = True
            for j in range(n):
                if i != j and check_collision(new_p, placements[j]):
                    valid = False
                    break
            
            if valid:
                old_p = placements[i]
                placements[i] = new_p
                new_side = compute_side(placements)
                if new_side <= current_side:
                    current_side = new_side
                    improved = True
                else:
                    placements[i] = old_p
        
        if not improved:
            break
    
    return center_placements(placements)

print("Compaction ready")

In [ ]:
# Shrink-wrap with binary search
def try_shrink(placements, target_side, max_repair=300):
    current_side = compute_side(placements)
    if current_side <= target_side:
        return placements if not has_any_collision(placements) else None
    
    scale = target_side / current_side
    scaled = scale_placements(placements, scale)
    repaired = repair_collisions(scaled, max_iter=max_repair)
    
    if has_any_collision(repaired):
        return None
    
    if compute_side(repaired) <= target_side * 1.02:
        return repaired
    return None

def shrink_wrap(placements, epsilon=0.001, max_iter=40):
    placements = [p.copy() for p in placements]
    
    if has_any_collision(placements):
        placements = repair_collisions(placements)
        if has_any_collision(placements):
            return placements
    
    current_side = compute_side(placements)
    n = len(placements)
    lo = theoretical_min_side(n)
    hi = current_side
    best = placements
    
    for _ in range(max_iter):
        if hi - lo < epsilon:
            break
        mid = (lo + hi) / 2
        result = try_shrink(best, mid)
        if result is not None:
            best = result
            hi = compute_side(result)
        else:
            lo = mid
    
    return center_placements(best)

print("Shrink-wrap ready")

In [ ]:
# Simulated Annealing with rotation optimization
def simulated_annealing(placements, max_iter=2000, T_init=1.0, T_min=0.001, cooling=0.997):
    placements = [p.copy() for p in placements]
    n = len(placements)
    if n < 2 or has_any_collision(placements):
        return placements
    
    current_side = compute_side(placements)
    best_placements = [p.copy() for p in placements]
    best_side = current_side
    
    T = T_init
    pos_step = min(TW, TH) * 0.15
    
    for _ in range(max_iter):
        if T < T_min:
            break
        
        i = np.random.randint(n)
        old_p = placements[i].copy()
        
        move_type = np.random.randint(4)
        
        if move_type == 0:  # Random position move
            placements[i].x += np.random.uniform(-pos_step, pos_step) * T
            placements[i].y += np.random.uniform(-pos_step, pos_step) * T
        elif move_type == 1:  # Flip rotation 180°
            placements[i].deg = (placements[i].deg + 180) % 360
        elif move_type == 2:  # Try key rotations
            placements[i].deg = np.random.choice([0, 90, 180, 270])
        else:  # Move toward center
            cx = sum(p.x for p in placements) / n
            cy = sum(p.y for p in placements) / n
            dx, dy = cx - placements[i].x, cy - placements[i].y
            dist = np.sqrt(dx*dx + dy*dy) + 1e-10
            placements[i].x += dx / dist * pos_step * T
            placements[i].y += dy / dist * pos_step * T
        
        # Check collision
        has_coll = False
        for j in range(n):
            if i != j and check_collision(placements[i], placements[j]):
                has_coll = True
                break
        
        if has_coll:
            placements[i] = old_p
            continue
        
        new_side = compute_side(placements)
        delta = new_side - current_side
        
        if delta < 0 or np.random.random() < np.exp(-delta / T):
            current_side = new_side
            if new_side < best_side:
                best_side = new_side
                best_placements = [p.copy() for p in placements]
        else:
            placements[i] = old_p
        
        T *= cooling
    
    return center_placements(best_placements)

print("Simulated annealing ready")

In [ ]:
# Rotation optimization pass
def optimize_rotations(placements, angles=[0, 45, 90, 135, 180, 225, 270, 315]):
    placements = [p.copy() for p in placements]
    n = len(placements)
    if n < 2:
        return placements
    
    improved = True
    while improved:
        improved = False
        current_side = compute_side(placements)
        
        for i in range(n):
            best_angle = placements[i].deg
            best_side = current_side
            
            for angle in angles:
                test_p = Placement(placements[i].x, placements[i].y, angle)
                
                valid = True
                for j in range(n):
                    if i != j and check_collision(test_p, placements[j]):
                        valid = False
                        break
                
                if not valid:
                    continue
                
                old_deg = placements[i].deg
                placements[i].deg = angle
                new_side = compute_side(placements)
                placements[i].deg = old_deg
                
                if new_side < best_side:
                    best_side = new_side
                    best_angle = angle
            
            if best_angle != placements[i].deg:
                placements[i].deg = best_angle
                improved = True
    
    return placements

print("Rotation optimization ready")

In [ ]:
# Multi-start solver with different strategies
def solve_n(n):
    """Solve for n trees using multiple strategies and keep the best."""
    
    if n == 1:
        return [Placement(0, 0, 0)], TH
    
    best = None
    best_side = float('inf')
    
    # Strategy pool
    strategies = []
    
    # 1. Optimal interlocking layout
    layout = optimal_interlocking_layout(n)
    layout = repair_collisions(layout, max_iter=300)
    if not has_any_collision(layout):
        strategies.append(layout)
    
    # 2. Checkerboard patterns with different spacing
    for sf in [1.0, 1.05, 1.1, 1.15, 1.2]:
        layout = checkerboard_layout(n, spacing_factor=sf)
        layout = repair_collisions(layout, max_iter=300)
        if not has_any_collision(layout):
            strategies.append(layout)
            break
    
    # 3. Tight hex patterns
    for sf in [0.8, 0.85, 0.9, 0.95, 1.0]:
        h_sp = TW * sf
        v_sp = TH * sf * 0.7
        cols = int(np.ceil(np.sqrt(n * v_sp / h_sp))) + 1
        
        placements = []
        row = 0
        idx = 0
        while idx < n:
            offset = h_sp * 0.5 if row % 2 == 1 else 0
            for col in range(cols):
                if idx >= n:
                    break
                deg = 180 if row % 2 == 1 else 0
                placements.append(Placement(col * h_sp + offset, row * v_sp, deg))
                idx += 1
            row += 1
        
        layout = center_placements(placements)
        layout = repair_collisions(layout, max_iter=300)
        if not has_any_collision(layout):
            strategies.append(layout)
            break
    
    # Evaluate and optimize each strategy
    for layout in strategies:
        if has_any_collision(layout):
            continue
        
        # Quick check
        side = compute_side(layout)
        if side >= best_side * 1.5:
            continue
        
        # Optimize
        optimized = layout
        
        # Compaction
        optimized = compact_greedy(optimized, iterations=100, step=0.01)
        
        # Rotation optimization (for smaller n)
        if n <= 50:
            optimized = optimize_rotations(optimized)
        
        # Shrink-wrap
        optimized = shrink_wrap(optimized, epsilon=0.001)
        
        # More compaction
        optimized = compact_toward_bbox_center(optimized, iterations=100, step=0.008)
        
        # SA refinement
        sa_iters = min(4000, 800 + n * 15)
        optimized = simulated_annealing(optimized, max_iter=sa_iters, T_init=0.8, cooling=0.998)
        
        # Final shrink-wrap
        optimized = shrink_wrap(optimized, epsilon=0.0005)
        
        # Final compaction
        optimized = compact_toward_bbox_center(optimized, iterations=150, step=0.005)
        
        side = compute_side(optimized)
        if side < best_side and not has_any_collision(optimized):
            best_side = side
            best = [p.copy() for p in optimized]
    
    # Fallback if nothing worked
    if best is None:
        layout = checkerboard_layout(n, spacing_factor=1.5)
        layout = repair_collisions(layout, max_iter=500)
        best = layout
        best_side = compute_side(best)
    
    return center_placements(best), best_side

print("Multi-strategy solver ready")

In [ ]:
# Solve all puzzles
MAX_N = 200

print("=" * 60)
print("SOLVING ALL PUZZLES")
print("=" * 60)

solutions = {}
sides = {}
start_time = time.time()

for n in range(1, MAX_N + 1):
    iter_start = time.time()
    placements, side = solve_n(n)
    iter_time = time.time() - iter_start
    
    # Final safety check
    if has_any_collision(placements):
        placements = repair_collisions(placements, max_iter=1000)
        if has_any_collision(placements):
            placements = scale_placements(placements, 1.5)
            placements = repair_collisions(placements, max_iter=500)
        side = compute_side(placements)
    
    solutions[n] = center_placements(placements)
    sides[n] = side
    
    if n <= 10 or n % 20 == 0:
        contrib = side**2 / n
        print(f"n={n:3d}: side={side:.4f}, contrib={contrib:.4f}, time={iter_time:.1f}s")

total_time = time.time() - start_time
print(f"\nTotal time: {total_time:.1f}s ({total_time/60:.1f} min)")

In [ ]:
# Validation
print("\n" + "=" * 60)
print("VALIDATION")
print("=" * 60)

failed = []
for n in range(1, MAX_N + 1):
    if has_any_collision(solutions[n]):
        failed.append(n)
        print(f"  n={n}: COLLISION DETECTED")

if failed:
    print(f"\nFailed puzzles: {failed}")
    print("Attempting aggressive repair...")
    for n in failed:
        solutions[n] = repair_collisions(solutions[n], max_iter=2000)
        if has_any_collision(solutions[n]):
            solutions[n] = scale_placements(solutions[n], 2.0)
            solutions[n] = repair_collisions(solutions[n], max_iter=1000)
        solutions[n] = center_placements(solutions[n])
        sides[n] = compute_side(solutions[n])
    
    still_failed = [n for n in failed if has_any_collision(solutions[n])]
    if still_failed:
        print(f"Still failed: {still_failed}")
    else:
        print("All repaired!")
else:
    print("ALL PUZZLES VALID - NO OVERLAPS!")

# Calculate final score
score = sum(sides[n]**2 / n for n in range(1, MAX_N + 1))
print(f"\nFINAL SCORE: {score:.2f}")

# Breakdown
print("\nScore breakdown:")
for n in [1, 2, 5, 10, 25, 50, 100, 200]:
    contrib = sides[n]**2 / n
    print(f"  n={n:3d}: side={sides[n]:.4f}, contrib={contrib:.4f}")

In [ ]:
# Create submission
rows = []
for n in range(1, MAX_N + 1):
    for i, p in enumerate(solutions[n]):
        rows.append({
            'id': f'{n:03d}_{i}',
            'x': f's{p.x:.6f}',
            'y': f's{p.y:.6f}',
            'deg': f's{p.deg:.6f}',
        })

submission = pd.DataFrame(rows)
submission.to_csv('submission.csv', index=False)

print(f"Saved submission.csv ({len(submission)} rows)")
print(f"\nExpected Score: {score:.2f}")
print("\nFirst 20 rows:")
print(submission.head(20))